In [1]:
from nilmtk import DataSet, MeterGroup
import warnings
warnings.filterwarnings("ignore")

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 29 days


In [2]:
ds = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")

In [3]:
from nilmtk.disaggregate import CombinatorialOptimisation

In [4]:
co = CombinatorialOptimisation()

In [5]:
building_num = 11

In [6]:
elec = ds.buildings[building_num].elec

Reducing time window

In [7]:
ds.set_window(start='2014-04-01 00:00:00', end='2014-05-01 00:00:00')

In [8]:
elec

MeterGroup(meters=
  ElecMeter(instance=1, building=11, dataset='WikiEnergy', site_meter, appliances=[])
  ElecMeter(instance=2, building=11, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=3, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=4, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=2)])
  ElecMeter(instance=5, building=11, dataset='WikiEnergy', appliances=[Appliance(type='dish washer', instance=1)])
  ElecMeter(instance=6, building=11, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])
  ElecMeter(instance=7, building=11, dataset='WikiEnergy', appliances=[Appliance(type='electric furnace', instance=1)])
  ElecMeter(instance=8, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=3)])
  ElecMeter(instance=9, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets

In [9]:
fridge_elecmeter = elec['fridge']

In [10]:
fridge_elecmeter

ElecMeter(instance=10, building=11, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=1)])

In [11]:
fridge_mg = MeterGroup([fridge_elecmeter])

In [12]:
co.train(fridge_mg)

Training model for submeter 'ElecMeter(instance=10, building=11, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=1)])'
Done training!


In [13]:
co.model

[{'states': array([  0, 139, 431], dtype=int32),
  'training_metadata': ElecMeter(instance=10, building=11, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=1)])}]

So, fridge is learnt as a 3 state appliance. What if we wanted to specify it to use 2 states? The latest version of nilmtk allows us to specify the number of states for an appliance.

In [14]:
num_states_dict = {fridge_elecmeter:2}

In [15]:
co = CombinatorialOptimisation()
co.train(fridge_mg, num_states_dict=num_states_dict)

Training model for submeter 'ElecMeter(instance=10, building=11, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=1)])'
Done training!


In [16]:
co.model

[{'states': array([  0, 143], dtype=int32),
  'training_metadata': ElecMeter(instance=10, building=11, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=1)])}]

Now, fridge is learnt as a 2 state appliance.

Let us try the same thing with FHMM now

In [17]:
from nilmtk.disaggregate import FHMM

In [18]:
f = FHMM()

In [19]:
f.train(fridge_mg)

Learnt states:  3
Training model for submeter 'ElecMeter(instance=10, building=11, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=1)])'


In [20]:
f.model.means_

array([[   3.02034344],
       [ 146.2202185 ],
       [ 176.49027576]])

In [21]:
f = FHMM()
f.train(fridge_mg, num_states_dict)

User specified  2
Training model for submeter 'ElecMeter(instance=10, building=11, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=1)])'


In [22]:
f.model.means_

array([[   0.        ],
       [ 144.87951888]])

So, now we have a 2 state learnt model for the Fridge.